In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# EmotiEffLib imports
from facial_analysis import EmotiEffLibRecognizer

In [ ]:
images_folder = '../data/CityInfant/BlackWhite/'

In [ ]:
df = pd.read_csv('..data/AU/pyafar_infants.csv')
df = df.iloc[:,[0,-2]]
df.to_csv("../data/CityInfant/Validationinfo/labels.csv", index=False)

In [ ]:
image_names = df['Image'].tolist()
affect_labels = df['imageCat'].tolist()

In [ ]:
# Split data 

X_train, X_val, y_train, y_val = train_test_split(image_names, affect_labels, test_size=0.2, random_state=42, stratify=numerical_labels) 